In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
#import altair as alt
import alpaca_trade_api as tradeapi
import requests
import json
import time
load_dotenv(dotenv_path="Api_keys.env")

True

In [2]:
key = os.getenv("alphavantage")
key

'UACKLUCYFFCOTJ55'

In [7]:
   # tickers to gather data for
# separated by industry
corp_list = [ "AAL","XOM", "PFE"]


for symbol in corp_list:
    # makes request to alphavantage to get fundamental data
    content = requests.get(url= "https://www.alphavantage.co/query?",  params = {"function":"BALANCE_SHEET", "symbol": symbol, "apikey": key})
    data = content.json()
data

{'Note': 'Thank you for using Alpha Vantage! Our standard API call frequency is 5 calls per minute and 500 calls per day. Please visit https://www.alphavantage.co/premium/ if you would like to target a higher API call frequency.'}

In [6]:
 # convert request json to dataframe
data_test = data['quarterlyReports']
df = pd.DataFrame.from_dict(data_test)
df.set_index('fiscalDateEnding', inplace=True)
file_name = f'Data/Airline/{symbol}_quarterly_reports.csv'
df.to_csv(file_name)
file_name


KeyError: 'quarterlyReports'

In [8]:
Tech_list = ["AAPL","AMZN","FB"]
for symbol in Tech_list:
    # makes request to alphavantage to get fundamental data
    content = requests.get(url= "https://www.alphavantage.co/query?",  params = {"function":"BALANCE_SHEET", "symbol": symbol, "apikey": key})
    data = content.json()

In [9]:
 # convert request json to dataframe
data_test = data['quarterlyReports']
df = pd.DataFrame.from_dict(data_test)
df.set_index('fiscalDateEnding', inplace=True)
file_name = f'Data/Tech/{symbol}_quarterly_reports.csv'
df.to_csv(file_name)
file_name

KeyError: 'quarterlyReports'

In [10]:
symbol_list = ["AAPL","AAL","AMZN", "FB","XOM"]

for symbol in symbol_list:
    stock_price = requests.get('https://www.alphavantage.co/query?', params={'function':'TIME_SERIES_MONTHLY_ADJUSTED','symbol':symbol,
                                                                              'apikey':'Alpha_vantage_Api_key'})
    stock_price = stock_price.json()
    try:
        stock_price = stock_price['Monthly Adjusted Time Series']
    except KeyError:
        print('KeyError. Trying again in 60 seconds...')  
        time.sleep(60)
        stock_price = requests.get('https://www.alphavantage.co/query?', params={'function':'TIME_SERIES_MONTHLY_ADJUSTED','symbol':symbol,
                                                                              'apikey':'Alpha_vantage_Api_key'})
        stock_price = stock_price['Monthly Adjusted Time Series']
    data = {}
    date_list = []
    close_list = []
    for date in stock_price:
        data[date] = stock_price[date]['4. close']
        date_list.append(date)
        close_list.append(stock_price[date]['4. close'])
    data = {'date': date_list, 'close': close_list}
    price_df = pd.DataFrame.from_dict(data)
    price_df.columns = ['date', 'close']
    price_df.set_index('date', inplace=True)
    price_df_ascending = price_df.sort_values(by="date", ascending=True)
    price_df_average = price_df_ascending.rolling(3).mean() 
    price_df_average = price_df_ascending.iloc[::3, :].truncate(before = "2019-03-29")
    final_month = price_df_ascending.iloc[-1] # we wanna append final_month to price_df_average
    price_df_average.append(final_month) 
    file_name = f'Data/Prices/{symbol}.csv'
    price_df.to_csv(file_name)
    print(f'{symbol}:')
    print(price_df_average)
    print('---')
    print(final_month)

FileNotFoundError: [Errno 2] No such file or directory: 'Data/Prices/AAPL.csv'